# Introducing a new pipettor

Procedure to add new pipettor from scratch, without any settings present.

In [1]:
import low_level_comm as llc
import cartesian
import tools
import calibration
import param
from importlib import reload 
import logging
import json
import racks

1. Install pipette rack; installed in slot 3, 0
2. Add pipette rack settings file into folder configs. Since all pipettor racks are the same, pipette_rack.ini file will be used. It should already exist, as p1000 was introduced before
3. Initialize rack for p200 pipette

In [2]:
p200_rack = racks.rack(rack_name='p200_tool_rack', rack_type='pipette_rack', x_slot=3, y_slot=0)

Right now, rack does not have any position information (as expected). Next step is to obtain this info.

In [3]:
p200_rack.rack_data

{'name': 'p200_tool_rack', 'n_x': 3, 'n_y': 0, 'type': 'pipette_rack'}

4. Performing newly added rack calibration

In [2]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [6]:
ar.home()

In [4]:
stp = tools.stationary_touch_probe(ar)

In [16]:
tp = tools.mobile_touch_probe.getTool(ar)

4a. Calibrating mobile touch probe against immobile one

In [17]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [18]:
ar.move(z=300)

4b. Calibrating p200 rack

In [19]:
xr, yr, zr = calibration.calibrateRack(tp, p200_rack)

In [20]:
p200_rack.rack_data

{'name': 'p200_tool_rack',
 'n_x': 3,
 'n_y': 0,
 'type': 'pipette_rack',
 'position': [539.89, 65.25, 445.95],
 'pos_stalagmyte': [92.0, 65.8, 491.35]}

In [21]:
p200_rack.save()

5. At this point, a new file, p200_tool_rack.json, appeared, and it now contains information about rack position and immobile probe position at the moment of calibration.

In [22]:
tp.returnTool()

6. Rack should be ready to use now.
7. Place p200 pipette into the rack
8. Create config file for the p200 pipette; p200_tool.ini
9. Pick up p200 tool to test it

In [8]:
p200 = tools.pipettor.getTool(robot=ar, tool_name='p200_tool')

In [14]:
p200.movePlunger(-1)

In [11]:
p200.home()

In [15]:
p200.returnTool()

10. measure height difference between mobile touch probe and the pipettor (may be approximate, it will be used to calculate proper Z for calibration.
11. In tool settings file (such as p200_tool.ini), modify delta_length_for_calibration parameter with resulted difference. If tool is longer than touch probe, the value is positive.

# Introducing p20 pipettor

In [1]:
import low_level_comm as llc
import cartesian
import tools
import calibration
import param
from importlib import reload 
import logging
import json
import racks

In [28]:
p20_rack = racks.rack(rack_name='p20_tool_rack', rack_type='pipette_rack', x_slot=4, y_slot=0)

In [29]:
p20_rack.rack_data

{'name': 'p20_tool_rack', 'n_x': 4, 'n_y': 0, 'type': 'pipette_rack'}

In [6]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [10]:
ar.home()

In [8]:
stp = tools.stationary_touch_probe(ar)

In [12]:
tp = tools.mobile_touch_probe.getTool(ar)

In [13]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [21]:
ar.move(z=300)

In [31]:
# To adjust height of calibration, modify xy_calibration_height parameter in pipette_rack.ini
# or a file for corresponding tool.
# Positive value means probe will go lower
xr, yr, zr = calibration.calibrateRack(tp, p20_rack)

In [32]:
tp.returnTool()

In [34]:
p20 = tools.pipettor.getTool(robot=ar, tool_name='p20_tool')

In [35]:
p20.returnTool()

## Testing tool calibration

In [36]:
p20 = tools.pipettor.getTool(robot=ar, tool_name='p20_tool')

In [37]:
calibration.calibrateTool(p20, stp)

(91.375, 66.95, 460.15)

In [38]:
p20.returnTool()